<a href="https://colab.research.google.com/github/GuiSSMartins/DTx-Project/blob/main/DTx_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from graphviz import Digraph # para representação gráfica de grafos direcionados (utiliza linguagem dot)
import matplotlib.image as mpimg # operações relacionadas a imagens
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc #importing garbage collector
import time

In [2]:
# função para reduzir a dimensão de um dataset
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

### A análise exploratória dos dados e as suas coneções passam a ser feitas no KNIME!!!!!!

## Decomposição da Série Temporal

In [3]:
df_loja1_serie_temporal_agregada = pd.read_csv('loja1.csv')

import numpy as np
import tensorflow as tf

df_loja1_serie_temporal_agregada = reduce_mem_usage(df_loja1_serie_temporal_agregada)

n_periods = 1913

# eliminar as colunas indesejadas para a construção da série temporal
df_time_series = pd.DataFrame(data={"Vendas": df_loja1_serie_temporal_agregada['Vendas_do_dia'].tolist()}, index=df_loja1_serie_temporal_agregada['date'])

Mem. usage decreased to  0.05 Mb (63.8% reduction)


In [4]:
df_loja1_serie_temporal_agregada

,date,wm_yr_wk,wday,month,year,d,evento,Vendas_do_dia,week_group
0,2011-01-29,11101,1,1,2011,d_1,0,4337,0
1,2011-01-30,11101,2,1,2011,d_2,0,4155,0
2,2011-01-31,11101,3,1,2011,d_3,0,2816,1
3,2011-02-01,11101,4,2,2011,d_4,0,3051,1
4,2011-02-02,11101,5,2,2011,d_5,0,2630,1
...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,11612,5,4,2016,d_1909,0,3722,1
1909,2016-04-21,11612,6,4,2016,d_1910,0,3709,1
1910,2016-04-22,11612,7,4,2016,d_1911,0,4387,1
1911,2016-04-23,11613,1,4,2016,d_1912,0,5577,0


In [5]:
print(df_loja1_serie_temporal_agregada.columns)

Index(['date', 'wm_yr_wk', 'wday', 'month', 'year', 'd', 'evento',
       'Vendas_do_dia', 'week_group'],
      dtype='object')


In [6]:
print(df_time_series)

            Vendas
date              
2011-01-29    4337
2011-01-30    4155
2011-01-31    2816
2011-02-01    3051
2011-02-02    2630
...            ...
2016-04-20    3722
2016-04-21    3709
2016-04-22    4387
2016-04-23    5577
2016-04-24    6113

[1913 rows x 1 columns]


## Carregar dados (agora concatenados)

In [7]:
df_loja1_RNN = reduce_mem_usage(df_loja1_serie_temporal_agregada)

Mem. usage decreased to  0.05 Mb (0.0% reduction)


# Treino de Modelos

##### Rede Neuronal Recurrente (RNN)

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

import numpy as np

# Select specific input and output columns
# Assuming df_loja1_RNN is your DataFrame
selected_input_columns = df_loja1_RNN.iloc[:, [1, 2, 7, 8, 6]]  # Access columns using .iloc method
selected_output_column = df_loja1_RNN.iloc[:, 7]  # Access single column directly

from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

print(selected_output_column.values)

# Fit scaler to the data and transform it
selected_input_columns = scaler.fit_transform(selected_input_columns)
selected_output_column = scaler.fit_transform(selected_output_column.values.reshape(-1, 1))

import tensorflow as tf
from tensorflow.keras.preprocessing import timeseries_dataset_from_array

# Assuming selected_input_columns and selected_output_column are arrays or lists
input_data = selected_input_columns[:-7]
targets = selected_output_column[7:]

# Assuming both input_data and targets are arrays or lists
sequence_length = 7
sequences = [(input_data[i:i+sequence_length], targets[i+sequence_length]) for i in range(len(input_data) - sequence_length)]

# Extract input sequences and targets
input_sequences = [seq[0] for seq in sequences]
target_values = [seq[1] for seq in sequences]

# Convert lists to TensorFlow tensors
input_sequences_tensor = tf.constant(input_sequences, dtype=tf.float32)
target_values_tensor = tf.constant(target_values, dtype=tf.float32)

# Create a TensorFlow dataset from tensors
dataset = tf.data.Dataset.from_tensor_slices((input_sequences_tensor, target_values_tensor))

# Shuffle and batch your dataset
batch_size = 32
buffer_size = len(sequences)
dataset = dataset.shuffle(buffer_size).batch(batch_size)

for batch in dataset:
    inputs, target = batch
    # Now you can use inputs and target for training
    break

[4337 4155 2816 ... 4387 5577 6113]


In [9]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 7, 5), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [10]:
inputs, targets = batch

len(inputs)

32

In [11]:
len(targets)

32

In [12]:
import tensorflow as tf

# Assuming 'dataset' is your BatchObject and 'train_split_ratio' is the ratio of data you want in the training set
def split_BatchDataset(dataset, train_split_ratio=0.8):
    # Get the total number of elements in the dataset
    total_elements = sum(1 for _ in dataset)

    # Calculate the number of elements for the training set
    train_elements = int(total_elements * train_split_ratio)

    # Create a dataset with train_elements elements for the training set
    train_dataset = dataset.take(train_elements)

    # Create a dataset with the remaining elements for the validation set
    test_dataset = dataset.skip(train_elements)

    return train_dataset, test_dataset

# Example usage
train_dataset, test_dataset = split_BatchDataset(dataset)

In [13]:
import numpy as np

# Inicializar listas para armazenar os dados
data_list = []
label_list = []

# Iterar sobre o conjunto de dados e extrair os dados e rótulos
for batch in train_dataset:
    data, label = batch  # Extrair os dados e rótulos da tupla
    data_list.append(data.numpy())  # Converter os dados para um array NumPy
    label_list.append(label.numpy())  # Converter os rótulos para um array NumPy

# Converter as listas de dados e rótulos para arrays NumPy
train_data_numpy = np.concatenate(data_list, axis=0)
train_labels_numpy = np.concatenate(label_list, axis=0)

In [20]:
# Assuming test_dataset is an instance of _SkipDataset
targets_test = []

# Iterate over the dataset to extract targets
for batch in test_dataset:
    inputs, target = batch  # Adjust this line according to the actual attribute/method name
    for value in target:
      targets_test.append(value)

targets_test = scaler.inverse_transform(targets_test)

print("Targets:", len(targets_test))

Targets: 363


In [24]:
hiperparameters = {
    'learning_rate': [0.1],
    'neurons': [128, 256],
    'dense_neurons': [10],
    'activation': ['relu'],
    'dropout_rate': [0.1]
}

In [31]:
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras import layers, models
from itertools import product
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Função para criar e treinar o modelo
def build_and_train_model(learning_rate, neurons, dense_neurons, activation, dropout_rate):
    model = models.Sequential()
    model.add(layers.LSTM(neurons, input_shape=(7, 5), return_sequences=True))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.LSTM(neurons, return_sequences=False))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(dense_neurons, activation))
    model.add(layers.Dense(1))  # Output layer

    optimizer = tf.optimizers.Adam(learning_rate)
    model.compile(optimizer=optimizer,
                  loss='mean_absolute_error',
                  metrics=[RootMeanSquaredError()])

    history = model.fit(train_data_numpy, train_labels_numpy, epochs=1000, validation_split=0.2, batch_size=batch_size, shuffle=True, callbacks = [early_stopping_callback, callbacks])

    predictions = model.predict(test_dataset)
    predictions = scaler.inverse_transform(predictions.reshape(-1, 1))

    mae = mean_absolute_error(targets_test, predictions)
    mse = mean_squared_error(targets_test, predictions)
    rmse = np.sqrt(mse)

    return [predictions, mae, mse, rmse]

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='ckpt/model_RNN_{epoch}_{val_loss:.3f}.hdf5', #path where to save the model
        save_best_only=True, #overwrite the current checkpoint if and only if
        monitor='val_loss', #the val_loss score has improved
        save_weights_only=True, #if True, only the weights are saved
        verbose=1, #verbosity mode
        period=5) # gravado callback por cada 5 epochs
]

# Define the EarlyStopping callback
early_stopping_callback = EarlyStopping(
    monitor='loss',    # Quantity to be monitored for improvement
    patience=20,            # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True   # Restores the model weights from the epoch with the best value of monitored quantity
)

#model.fit(train_data_numpy, train_labels_numpy, epochs=1000, validation_split=0.2, batch_size=batch_size, shuffle=True)

In [32]:
import math

best_RNN = [[], 0, 0, math.inf];

# Iterando sobre todas as combinações de hiperparâmetros
for learning_rate, neurons, dense_neurons, activation, dropout_rate in product(*hiperparameters.values()):
    print(f"Treinando modelo com learning_rate={learning_rate}, num_neurons={neurons}, num_dense_neurons={dense_neurons}, funcao_de_ativacao={activation}, dropout_rate={dropout_rate}")
    [predictions, mae, mse, rmse] = build_and_train_model(learning_rate, neurons, dense_neurons, activation, dropout_rate)
    print(f'MAE:{mae}, MSE:{mse} RMSE: {rmse}')

    if (best_RNN[3] > rmse):
      best_RNN = [predictions, mae, mse, rmse]

Treinando modelo com learning_rate=0.01, num_neurons=128, num_dense_neurons=10, funcao_de_ativacao=relu, dropout_rate=0.1
Epoch 1/1000
39/39 [==============================] - 5s 30ms/step - loss: 0.6176 - root_mean_squared_error: 0.8164 - val_loss: 0.4238 - val_root_mean_squared_error: 0.5678
Epoch 2/1000
39/39 [==============================] - 1s 15ms/step - loss: 0.4292 - root_mean_squared_error: 0.5688 - val_loss: 0.3729 - val_root_mean_squared_error: 0.4838
Epoch 3/1000
39/39 [==============================] - 1s 16ms/step - loss: 0.4020 - root_mean_squared_error: 0.5286 - val_loss: 0.3821 - val_root_mean_squared_error: 0.4875
Epoch 4/1000
39/39 [==============================] - 1s 15ms/step - loss: 0.3929 - root_mean_squared_error: 0.5145 - val_loss: 0.3609 - val_root_mean_squared_error: 0.4727
Epoch 5/1000
37/39 [===========================>..] - ETA: 0s - loss: 0.3859 - root_mean_squared_error: 0.5100
Epoch 5: val_loss improved from inf to 0.38260, saving model to ckpt/model_

MAE:1072.9980766975013, MSE:1773213.848431672 RMSE: 1331.6207599882453
Treinando modelo com learning_rate=0.01, num_neurons=128, num_dense_neurons=10, funcao_de_ativacao=relu, dropout_rate=0.2
Epoch 1/1000
39/39 [==============================] - 28s 645ms/step - loss: 0.5659 - root_mean_squared_error: 0.7526 - val_loss: 0.4280 - val_root_mean_squared_error: 0.5439
Epoch 2/1000
39/39 [==============================] - 1s 27ms/step - loss: 0.4186 - root_mean_squared_error: 0.5471 - val_loss: 0.3599 - val_root_mean_squared_error: 0.4669
Epoch 3/1000
38/39 [============================>.] - ETA: 0s - loss: 0.3980 - root_mean_squared_error: 0.5170
Epoch 3: val_loss did not improve from 0.34799
39/39 [==============================] - 1s 20ms/step - loss: 0.3963 - root_mean_squared_error: 0.5152 - val_loss: 0.3830 - val_root_mean_squared_error: 0.4972
Epoch 4/1000
39/39 [==============================] - 1s 15ms/step - loss: 0.4011 - root_mean_squared_error: 0.5210 - val_loss: 0.3810 - val_

MAE:1005.4315205879292, MSE:1643169.0253392328 RMSE: 1281.8615468681603
Treinando modelo com learning_rate=0.01, num_neurons=128, num_dense_neurons=20, funcao_de_ativacao=relu, dropout_rate=0.1
Epoch 1/1000
39/39 [==============================] - 6s 30ms/step - loss: 0.5847 - root_mean_squared_error: 0.7778 - val_loss: 0.3673 - val_root_mean_squared_error: 0.4742
Epoch 2/1000
38/39 [============================>.] - ETA: 0s - loss: 0.4201 - root_mean_squared_error: 0.5405
Epoch 2: val_loss did not improve from 0.33862
39/39 [==============================] - 1s 15ms/step - loss: 0.4206 - root_mean_squared_error: 0.5405 - val_loss: 0.3885 - val_root_mean_squared_error: 0.5172
Epoch 3/1000
39/39 [==============================] - 1s 16ms/step - loss: 0.3923 - root_mean_squared_error: 0.5123 - val_loss: 0.3851 - val_root_mean_squared_error: 0.4932
Epoch 4/1000
39/39 [==============================] - 1s 15ms/step - loss: 0.3919 - root_mean_squared_error: 0.5205 - val_loss: 0.3731 - val_r

MAE:1066.210762807333, MSE:1744028.717351331 RMSE: 1320.6167942864163
Treinando modelo com learning_rate=0.01, num_neurons=128, num_dense_neurons=20, funcao_de_ativacao=relu, dropout_rate=0.2
Epoch 1/1000
39/39 [==============================] - 4s 30ms/step - loss: 0.5514 - root_mean_squared_error: 0.7245 - val_loss: 0.3717 - val_root_mean_squared_error: 0.4785
Epoch 2/1000
37/39 [===========================>..] - ETA: 0s - loss: 0.4203 - root_mean_squared_error: 0.5441
Epoch 2: val_loss did not improve from 0.33512
39/39 [==============================] - 1s 16ms/step - loss: 0.4236 - root_mean_squared_error: 0.5536 - val_loss: 0.4301 - val_root_mean_squared_error: 0.5559
Epoch 3/1000
39/39 [==============================] - 1s 15ms/step - loss: 0.4109 - root_mean_squared_error: 0.5390 - val_loss: 0.3833 - val_root_mean_squared_error: 0.5021
Epoch 4/1000
39/39 [==============================] - 1s 16ms/step - loss: 0.3957 - root_mean_squared_error: 0.5162 - val_loss: 0.4029 - val_roo

MAE:1063.5700319033542, MSE:1832260.8923239557 RMSE: 1353.610317751736
Treinando modelo com learning_rate=0.01, num_neurons=256, num_dense_neurons=10, funcao_de_ativacao=relu, dropout_rate=0.1
Epoch 1/1000
39/39 [==============================] - 6s 46ms/step - loss: 0.7147 - root_mean_squared_error: 0.9430 - val_loss: 0.6618 - val_root_mean_squared_error: 0.8647
Epoch 2/1000
39/39 [==============================] - 1s 23ms/step - loss: 0.6029 - root_mean_squared_error: 0.7961 - val_loss: 0.6322 - val_root_mean_squared_error: 0.8145
Epoch 3/1000
39/39 [==============================] - 1s 16ms/step - loss: 0.5531 - root_mean_squared_error: 0.7328 - val_loss: 0.5449 - val_root_mean_squared_error: 0.7176
Epoch 4/1000
39/39 [==============================] - 1s 15ms/step - loss: 0.4944 - root_mean_squared_error: 0.6541 - val_loss: 0.4759 - val_root_mean_squared_error: 0.6358
Epoch 5/1000
38/39 [============================>.] - ETA: 0s - loss: 0.4719 - root_mean_squared_error: 0.6178
Epoc

...

MAE:783.8412741499113, MSE:1005483.9344114356 RMSE: 1002.7382182860268
Treinando modelo com learning_rate=0.1, num_neurons=256, num_dense_neurons=20, funcao_de_ativacao=relu, dropout_rate=0.2
Epoch 1/1000
39/39 [==============================] - 4s 29ms/step - loss: 1.3396 - root_mean_squared_error: 3.4868 - val_loss: 0.7801 - val_root_mean_squared_error: 1.0287
Epoch 2/1000
39/39 [==============================] - 1s 15ms/step - loss: 0.7948 - root_mean_squared_error: 1.0091 - val_loss: 0.7800 - val_root_mean_squared_error: 1.0213
Epoch 3/1000
36/39 [==========================>...] - ETA: 0s - loss: 0.7995 - root_mean_squared_error: 1.0064
Epoch 3: val_loss did not improve from 0.32682
39/39 [==============================] - 1s 16ms/step - loss: 0.7965 - root_mean_squared_error: 1.0079 - val_loss: 0.7865 - val_root_mean_squared_error: 1.0474
Epoch 4/1000
39/39 [==============================] - 1s 15ms/step - loss: 0.8037 - root_mean_squared_error: 1.0143 - val_loss: 0.7801 - val_roo

MAE:988.1851897458613, MSE:1476226.0897013124 RMSE: 1215.0004484366714
Treinando modelo com learning_rate=0.1, num_neurons=512, num_dense_neurons=10, funcao_de_ativacao=relu, dropout_rate=0.1
Epoch 1/1000
37/39 [===========================>..] - ETA: 0s - loss: 2.6671 - root_mean_squared_error: 5.3617
Epoch 1: val_loss did not improve from 0.32682
39/39 [==============================] - 5s 33ms/step - loss: 2.5998 - root_mean_squared_error: 5.2683 - val_loss: 0.7845 - val_root_mean_squared_error: 1.0135
Epoch 2/1000
39/39 [==============================] - 1s 16ms/step - loss: 0.7959 - root_mean_squared_error: 1.0062 - val_loss: 0.7812 - val_root_mean_squared_error: 1.0344
Epoch 3/1000
39/39 [==============================] - 1s 17ms/step - loss: 0.7956 - root_mean_squared_error: 1.0092 - val_loss: 0.7808 - val_root_mean_squared_error: 1.0332
Epoch 4/1000
39/39 [==============================] - 1s 16ms/step - loss: 0.8007 - root_mean_squared_error: 1.0184 - val_loss: 0.7909 - val_roo

MAE:783.8224020123425, MSE:1005140.1571664129 RMSE: 1002.5667843921485
Treinando modelo com learning_rate=0.1, num_neurons=512, num_dense_neurons=10, funcao_de_ativacao=relu, dropout_rate=0.2
Epoch 1/1000
39/39 [==============================] - 4s 32ms/step - loss: 2.5276 - root_mean_squared_error: 5.9332 - val_loss: 0.8138 - val_root_mean_squared_error: 1.0887
Epoch 2/1000
36/39 [==========================>...] - ETA: 0s - loss: 0.8058 - root_mean_squared_error: 1.0194
Epoch 2: val_loss did not improve from 0.32682
39/39 [==============================] - 1s 18ms/step - loss: 0.8029 - root_mean_squared_error: 1.0174 - val_loss: 0.7899 - val_root_mean_squared_error: 1.0539
Epoch 3/1000
39/39 [==============================] - 1s 17ms/step - loss: 0.7935 - root_mean_squared_error: 1.0068 - val_loss: 0.7804 - val_root_mean_squared_error: 1.0196
Epoch 4/1000
39/39 [==============================] - 1s 16ms/step - loss: 0.7968 - root_mean_squared_error: 1.0111 - val_loss: 0.7863 - val_roo

MAE:1015.6259614402658, MSE:1566115.272550904 RMSE: 1251.4452734941724
Treinando modelo com learning_rate=0.1, num_neurons=512, num_dense_neurons=20, funcao_de_ativacao=relu, dropout_rate=0.1
Epoch 1/1000
39/39 [==============================] - 7s 42ms/step - loss: 1.9974 - root_mean_squared_error: 6.9529 - val_loss: 0.8037 - val_root_mean_squared_error: 1.0742
Epoch 2/1000
39/39 [==============================] - 1s 18ms/step - loss: 0.7982 - root_mean_squared_error: 1.0122 - val_loss: 0.7830 - val_root_mean_squared_error: 1.0394
Epoch 3/1000
36/39 [==========================>...] - ETA: 0s - loss: 0.8013 - root_mean_squared_error: 1.0123
Epoch 3: val_loss did not improve from 0.32682
39/39 [==============================] - 1s 17ms/step - loss: 0.7943 - root_mean_squared_error: 1.0079 - val_loss: 0.7919 - val_root_mean_squared_error: 1.0571
Epoch 4/1000
39/39 [==============================] - 1s 17ms/step - loss: 0.7994 - root_mean_squared_error: 1.0158 - val_loss: 0.7803 - val_roo

MAE:783.8411900794482, MSE:1005482.3946479664 RMSE: 1002.7374505063458
Treinando modelo com learning_rate=0.1, num_neurons=512, num_dense_neurons=20, funcao_de_ativacao=relu, dropout_rate=0.2
Epoch 1/1000
39/39 [==============================] - 4s 32ms/step - loss: 2.8122 - root_mean_squared_error: 11.9585 - val_loss: 0.7800 - val_root_mean_squared_error: 1.0272
Epoch 2/1000
39/39 [==============================] - 1s 17ms/step - loss: 0.7967 - root_mean_squared_error: 1.0065 - val_loss: 0.7811 - val_root_mean_squared_error: 1.0341
Epoch 3/1000
39/39 [==============================] - 1s 17ms/step - loss: 0.7956 - root_mean_squared_error: 1.0057 - val_loss: 0.7944 - val_root_mean_squared_error: 1.0608
Epoch 4/1000
39/39 [==============================] - 1s 17ms/step - loss: 0.7967 - root_mean_squared_error: 1.0112 - val_loss: 0.7801 - val_root_mean_squared_error: 1.0299
Epoch 5/1000
38/39 [============================>.] - ETA: 0s - loss: 0.7959 - root_mean_squared_error: 1.0074
Epoc

In [33]:
best_RNN

[array([[3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],
        [3938.576],


In [ ]:
# Imprima os resultados
print("Melhores parâmetros: ", grid_result.best_params_)
print("Melhor precisão: ", grid_result.best_score_)

In [ ]:
# Make predictions
predictions = model.predict(test_dataset)
predictions = scaler.inverse_transform(predictions.reshape(-1, 1))

print("Predictions:")
print(len(predictions))

In [ ]:
# Flatten the array of arrays into a single list
flat_data = [item for sublist in predictions for item in sublist]

# Count the frequency of each item
unique_items, frequencies = np.unique(flat_data, return_counts=True)

#print(unique_items)
#print(frequencies)

# Assuming you have a list of data points called 'data'
plt.hist(predictions, bins='auto', edgecolor='black')  # 'auto' for automatic bin size determination
plt.xlabel('Valor da Venda')
plt.ylabel('Frequency')
plt.title('Previsões das Vendas')
plt.grid(True)  # Add grid lines
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(40, 6))

# Criar o gráfico de linhas
plt.plot(predictions[:30], label='Modelo', color='blue', linestyle='-', marker='o')
plt.plot(targets_test[:30], label='Valor verdadeiro', color='red', linestyle='--', marker='x')

# Adicionar título e rótulos dos eixos
plt.title('Gráfico de Duas Linhas')
plt.xlabel('Índices dos Elementos')
plt.ylabel('Valores')

# Adicionar legenda
plt.legend()

# Exibir o gráfico
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(targets_test, predictions)

# MSE calculation
mse = mean_squared_error(targets_test, predictions)

# RMSE calculation (square root of MSE)
rmse = np.sqrt(mse)

In [ ]:
mae

In [ ]:
mse

In [ ]:
rmse

In [ ]:
sum(targets_test - predictions)/len(predictions)

### Monte-Carlo dropout

In [ ]:
# Make predictions with Monte Carlo Dropout
def predict_with_mc_dropout(model, dataset, num_samples):
    preds = []

    for _ in range(num_samples):
        predictions = model.predict(dataset)
        preds.append(predictions)

    return preds

# Example usage: assuming you have a dataset called 'test_dataset' for testing
num_samples = 250
preds = predict_with_mc_dropout(model, test_dataset, num_samples)
preds = [scaler.inverse_transform(preds[i].reshape(-1, 1)) for i in range(len(preds))]

In [ ]:
len(preds)

In [ ]:
[sublist[0] for sublist in preds]

In [ ]:
mean_preds = np.mean(preds, axis=0)
var_preds = np.var(preds, axis=0)
sd_preds = np.sqrt(var_preds)

In [ ]:
len(mean_preds)

In [ ]:
len(sd_preds)

In [ ]:
#sd_preds

In [ ]:
sum(targets_test - mean_preds)/len(mean_preds)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(40, 6))

# Perform element-wise addition and subtraction
upper_bound = np.stack([mean + 1.96*sd for mean, sd in zip(mean_preds[:100], sd_preds[:100])]).ravel()
lower_bound = np.stack([mean - 1.96*sd for mean, sd in zip(mean_preds[:100], sd_preds[:100])]).ravel()

# Criar o gráfico de linhas
plt.plot(mean_preds[:100], label='Média do Modelo', color='blue', linestyle='-', marker='o')
plt.fill_between(range(100), lower_bound, upper_bound, color='blue', alpha=0.2, label='Intervalo 95% Confiança')
plt.plot(targets_test[:100], label='Valor verdadeiro', color='red', linestyle='--', marker='x')

# Adicionar título e rótulos dos eixos
plt.title('Gráfico de Duas Linhas')
plt.xlabel('Índices dos Elementos')
plt.ylabel('Valores')

# Adicionar legenda
plt.legend()

# Exibir o gráfico
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(60, 6))

# Perform element-wise addition and subtraction
upper_bound = np.stack([mean + 1.96*sd for mean, sd in zip(mean_preds, sd_preds)]).ravel()
lower_bound = np.stack([mean - 1.96*sd for mean, sd in zip(mean_preds, sd_preds)]).ravel()

# Criar o gráfico de linhas
plt.plot(mean_preds, label='Média do Modelo', color='blue', linestyle='-', marker='o')
plt.fill_between(range(363), lower_bound, upper_bound, color='blue', alpha=0.2, label='Intervalo 95% Confiança')
plt.plot(targets_test, label='Valor verdadeiro', color='red', linestyle='--', marker='x')

# Adicionar título e rótulos dos eixos
plt.title('Gráfico de Duas Linhas')
plt.xlabel('Índices dos Elementos')
plt.ylabel('Valores')

# Adicionar legenda
plt.legend()

# Exibir o gráfico
plt.grid(True)
plt.show()

### **Probabilistic** Bayesian Neural Network

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [ ]:
import keras
from keras import layers
import tensorflow as tf
import tensorflow_probability as tfp

hidden_units = [50, 50]
learning_rate = 0.001

def create_probablistic_bnn_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    features = inputs
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / len(train_data_numpy),
            activation="sigmoid",
        )(features)

    # Create a probabilistic output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

#'''
def run_experiment(model, loss, train_data_numpy, train_labels_numpy, test_dataset, num_epochs, learning_rate):
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_data_numpy, train_labels_numpy, epochs=num_epochs, validation_split=0.2)
    print("Model training finished.")

    print("Evaluating model performance on training dataset...")
    train_loss, train_rmse = model.evaluate(train_data_numpy, train_labels_numpy, verbose=0)
    print(f"Train RMSE: {round(train_rmse, 3)}")

    print("Evaluating model performance on test dataset...")
    test_loss, test_rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(test_rmse, 3)}")
#'''
# Assuming train_data_numpy, train_labels_numpy, and test_dataset are defined elsewhere
input_shape = (7, 5)  # Example shape, replace with actual shape
prob_bnn_model = create_probablistic_bnn_model(input_shape)

'''
def negative_loglikelihood(targets, estimated_distribution, log_variance):
    return -estimated_distribution.log_prob(targets, log_variance)
'''

def negative_log_likelihood(y_true, mean_log_variance):
    mean, log_variance = mean_log_variance[:, 0], mean_log_variance[:, 1]
    nll_loss = 0.5 * (log_variance + tf.square(y_true - mean) / tf.exp(log_variance)) + 0.5 * tf.math.log(2 * np.pi)
    return tf.reduce_mean(nll_loss)

num_epochs = 200
learning_rate = 0.001
loss_fn = tf.keras.losses.MeanSquaredError()

run_experiment(prob_bnn_model, negative_log_likelihood, train_data_numpy, train_labels_numpy, test_dataset, num_epochs, learning_rate)

In [ ]:
prediction_distribution = prob_bnn_model.predict(test_dataset)
# Reshape prediction_distribution to remove the extra dimension
prediction_distribution = np.squeeze(prediction_distribution)

# Now, you can use inverse_transform()
prediction_distribution = scaler.inverse_transform(prediction_distribution)
prediction_mean = np.mean(prediction_distribution, axis=1)
prediction_stdv = np.std(prediction_distribution, axis=1)

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

In [ ]:
prediction_distribution

In [ ]:
len(prediction_distribution[0])

In [ ]:
prediction_stdv

In [ ]:
upper, lower

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(40, 6))

# Criar o gráfico de linhas
plt.plot(prediction_mean[:100], label='Média do Modelo', color='blue', linestyle='-', marker='o')
plt.fill_between(range(100), lower[:100], upper[:100], color='blue', alpha=0.2, label='Intervalo Confiança 95%')
plt.plot(targets_test[:100], label='Valor verdadeiro', color='red', linestyle='--', marker='x')

# Adicionar título e rótulos dos eixos
plt.title('Gráfico de Duas Linhas')
plt.xlabel('Índices dos Elementos')
plt.ylabel('Valores')

# Adicionar legenda
plt.legend()

# Exibir o gráfico
plt.grid(True)
plt.show()

### Análises dos dados

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(df_time_series, model='additive', period=365)
decomposition.plot()
plt.show()

In [ ]:
# Represenção do digrafo, aplicando cores e percentagens nas arrestas
dot = Digraph(comment='Grafo para descrever a Estrutura do Dados')

dot.node('A', f'Total de estados', color='lightblue2', style='filled')
dot.node('B', f'Estado CA', color='lightblue2', style='filled')
dot.node('C', f'Estado TX', color='lightblue2', style='filled')
dot.node('D', f'Estado W', color='lightblue2', style='filled')
dot.node('E', f'LOJA 1', color='lightblue2', style='filled')
dot.node('F', f'LOJA 2', color='lightblue2', style='filled')
dot.node('G', f'LOJA 3', color='lightblue2', style='filled')
dot.node('H', f'LOJA 4', color='lightblue2', style='filled')
dot.node('I', f'LOJA 1', color='lightblue2', style='filled')
dot.node('J', f'LOJA 2', color='lightblue2', style='filled')
dot.node('K', f'LOJA 3', color='lightblue2', style='filled')
dot.node('L', f'LOJA 1', color='lightblue2', style='filled')
dot.node('M', f'LOJA 2', color='lightblue2', style='filled')
dot.node('N', f'LOJA 3', color='lightblue2', style='filled')
dot.node('JA', f'CATEGORY HOBBIES', color='lightblue2', style='filled')
dot.node('JB', f'CATEGORY FOOD', color='lightblue2', style='filled')
dot.node('JC', f'CATEGORY HOUSEHOLD', color='lightblue2', style='filled')
dot.node('JAA', f'DEPARTAMENT HOBBIES 1', color='lightblue2', style='filled')
dot.node('JAB', f'DEPARTAMENT HOBBIES 2', color='lightblue2', style='filled')
dot.node('JBA', f'DEPARTAMENT FOOD 1', color='lightblue2', style='filled')
dot.node('JBB', f'DEPARTAMENT FOOD 2', color='lightblue2', style='filled')
dot.node('JBC', f'DEPARTAMENT FOOD 3', color='lightblue2', style='filled')
dot.node('JCA', f'DEPARTAMENT HOUSEHOLD 1', color='lightblue2', style='filled')
dot.node('JCB', f'DEPARTAMENT HOUSEHOLD 2', color='lightblue2', style='filled')


dot.edge('A', 'B')
dot.edge('A', 'C')
dot.edge('A', 'D')
dot.edge('B', 'E')
dot.edge('B', 'F')
dot.edge('B', 'G')
dot.edge('B', 'H')
dot.edge('C', 'I')
dot.edge('C', 'J')
dot.edge('C', 'K')
dot.edge('D', 'L')
dot.edge('D', 'M')
dot.edge('D', 'N')
dot.edge('J', 'JA')
dot.edge('J', 'JB')
dot.edge('J', 'JC')
dot.edge('JA', 'JAA')
dot.edge('JA', 'JAB')
dot.edge('JB', 'JBA')
dot.edge('JB', 'JBB')
dot.edge('JB', 'JBC')
dot.edge('JC', 'JCA')
dot.edge('JC', 'JCB')

dot.render('/content/drive/MyDrive/Save/patient_dot', format='png', cleanup=True)

In [ ]:
image = mpimg.imread('/content/drive/MyDrive/Save/patient_dot.png')

fig, ax = plt.subplots() #criar uma subparcela única
ax.imshow(image) #exibir a imagem na subparcela e desativar os eixos
ax.axis('off')
#centralizar a imagem na subparcela
ax.set_position([0, 0, 1, 1])  # Define a posição da subparcela para cobrir toda a figura
#fig.savefig("/content/drive/MyDrive/Save/patient_dot.png")
plt.show() #pode nem ser colocado